In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/draft-safety/sample_submission.csv
/kaggle/input/draft-safety/test_safety/test_safety/img_2104.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_2121.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_2085.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_1895.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_1908.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_2046.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_2245.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_2189.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_1966.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_1923.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_1996.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_2105.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_2077.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_2130.jpg
/kaggle/input/draft-safety/test_safety/test_safety/img_

In [7]:
import tensorflow as tf
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define labels
labels = ['person', 'red_hat', 'yellow_hat', 'blue_hat', 'vest', 'white_hat']
num_classes = len(labels)

# Load ConvNeXtBase without the top layer
base_model = ConvNeXtBase(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze ConvNeXt layers

# Define custom head
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='sigmoid')(x)  # Sigmoid for multi-label classification

# Create model
model = Model(inputs, outputs, name="ConvNeXt_MultiLabel")

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print summary
model.summary()


Model: "ConvNeXt_MultiLabel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_23 (InputLayer)          │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ convnext_base (Functional)           │ (None, 7, 7, 1024)          │      87,566,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 88,094,342 (336.05 MB)

 Trainable params: 527,878 (2.01 MB)

 Non-trainable params: 87,566,464 (334.04 MB)

In [8]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split

# Load dataset
train_annotations = pd.read_csv('/kaggle/input/draft-safety/train/train_annotation.csv')

# Define possible labels
labels = ['person', 'red_hat', 'yellow_hat', 'blue_hat', 'vest', 'white_hat']

# Multi-label binarization
train_annotations['labels'] = train_annotations['labels'].apply(lambda x: x.split())
for label in labels:
    train_annotations[label] = train_annotations['labels'].apply(lambda x: 1 if label in x else 0)

# Split dataset into train and validation sets
train_df, val_df = train_test_split(train_annotations, test_size=0.2, random_state=42)
train_df["image_id"] = train_df["image_id"].astype(str) + ".jpg"
val_df["image_id"] = val_df["image_id"].astype(str) + ".jpg"

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Define data generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory="/kaggle/input/draft-safety/train/train_safety",
    x_col="image_id",
    y_col=labels,
    target_size=(300, 300),
    batch_size=16,
    class_mode="raw"
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    directory="/kaggle/input/draft-safety/train/train_safety",
    x_col="image_id",
    y_col=labels,
    target_size=(300, 300),
    batch_size=16,
    class_mode="raw"
)

# Load ConvNeXt model
base_model = ConvNeXtBase(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
base_model.trainable = False  # Freeze base layers

# Build model
inputs = tf.keras.Input(shape=(300, 300, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(len(labels), activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs, name="ConvNeXt_MultiLabel")

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
]

# Train model
history = model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=callbacks)

# Fine-tune model (unfreeze base layers)
base_model.trainable = True
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
history_fine = model.fit(train_generator, epochs=5, validation_data=val_generator, callbacks=callbacks)

print("Training complete!")

Found 1488 validated image filenames.
Found 372 validated image filenames.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


93/93 ━━━━━━━━━━━━━━━━━━━━ 83s 621ms/step - accuracy: 0.9602 - loss: 0.5921 - val_accuracy: 0.9946 - val_loss: 0.4881 - learning_rate: 0.0010
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 47s 474ms/step - accuracy: 0.9887 - loss: 0.4963 - val_accuracy: 0.9946 - val_loss: 0.4663 - learning_rate: 0.0010
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 48s 481ms/step - accuracy: 0.9926 - loss: 0.4861 - val_accuracy: 0.9946 - val_loss: 0.4509 - learning_rate: 0.0010
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 47s 473ms/step - accuracy: 0.9909 - loss: 0.4766 - val_accuracy: 0.9946 - val_loss: 0.4651 - learning_rate: 0.0010
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 47s 471ms/step - accuracy: 0.9937 - loss: 0.4656 - val_accuracy: 0.9973 - val_loss: 0.4393 - learning_rate: 0.0010
Epoch 6/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 48s 476ms/step - accuracy: 0.9930 - loss: 0.4553 - val_accuracy: 0.9973 - val_loss: 0.4233 - learning_rate: 0.0010
Epoch 7/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 47s 480ms/step - accuracy: 0.9919 - loss: 0.4622 - val_

In [9]:
# Unfreeze last 30 layers of ConvNeXt
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Freeze everything except last 30 layers
    layer.trainable = False

# Recompile with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(train_generator, epochs=5, validation_data=val_generator, callbacks=callbacks)


Epoch 1/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 82s 553ms/step - accuracy: 0.9442 - loss: 0.1946 - val_accuracy: 0.8575 - val_loss: 0.1542 - learning_rate: 1.0000e-05
Epoch 2/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 48s 487ms/step - accuracy: 0.9170 - loss: 0.1767 - val_accuracy: 0.8360 - val_loss: 0.1451 - learning_rate: 1.0000e-05
Epoch 3/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 48s 479ms/step - accuracy: 0.8937 - loss: 0.1569 - val_accuracy: 0.8441 - val_loss: 0.1431 - learning_rate: 1.0000e-05
Epoch 4/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 48s 484ms/step - accuracy: 0.9031 - loss: 0.1597 - val_accuracy: 0.8172 - val_loss: 0.1515 - learning_rate: 1.0000e-05
Epoch 5/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 48s 482ms/step - accuracy: 0.9037 - loss: 0.1598 - val_accuracy: 0.8199 - val_loss: 0.1377 - learning_rate: 1.0000e-05
Restoring model weights from the end of the best epoch: 5.


In [ ]:
# there is a probleme with this code
test_dir = "/kaggle/input/draft-safety/test_safety/test_safety/"
test_images = os.listdir(test_dir)
test_df = pd.DataFrame({"image_id": test_images})

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_dir,
    x_col="image_id",
    target_size=(300, 300),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

# Perform TTA (5 augmentations per image)
tta_steps = 5
predictions = np.zeros((len(test_generator), len(labels)))

for i in range(tta_steps):
    preds = model.predict(test_generator)
    predictions += preds

predictions /= tta_steps  # Average the predictions

# Convert predictions to labels
threshold = 0.5
predicted_labels = [
    " ".join([labels[i] for i, p in enumerate(pred) if p > threshold])
    for pred in predictions
]

# Save Submission
submission = pd.DataFrame({"image_id": test_images, "labels": predicted_labels})
submission.to_csv('submission.csv', index=False)
print("Submission saved successfully!")

In [10]:
# Paths
test_dir = "/kaggle/input/draft-safety/test_safety/test_safety/"
test_images = os.listdir(test_dir)
test_df = pd.DataFrame({"image_id": test_images})

# Image Generator
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_dir,
    x_col="image_id",
    target_size=(300, 300),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

# Perform TTA (5 augmentations per image)
tta_steps = 5
num_samples = len(test_df)  # Total test images
num_classes = len(labels)   # Number of labels

predictions = np.zeros((num_samples, num_classes))  # Correct shape

for i in range(tta_steps):
    preds = model.predict(test_generator, verbose=1)
    predictions += preds[:num_samples]  # Ensure correct shape

predictions /= tta_steps  # Average the predictions

# Convert predictions to labels
threshold = 0.5
predicted_labels = [
    " ".join([labels[i] for i, p in enumerate(pred) if p > threshold])
    for pred in predictions
]

# Save Submission
submission = pd.DataFrame({"image_id": test_images, "labels": predicted_labels})
submission.to_csv('submission.csv', index=False)
print("✅ Submission saved successfully!")


Found 400 validated image filenames.
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 628ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 318ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 325ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 324ms/step
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 326ms/step
✅ Submission saved successfully!


In [11]:
submission_df = pd.read_csv("submission.csv")
print("Unique image count in submission:", submission_df["image_id"].nunique())
print("Expected image count:", len(os.listdir("/kaggle/input/draft-safety/test_safety/test_safety")))

submission = pd.DataFrame({"image_id": [img.split('.')[0] for img in test_images], "labels": predicted_labels})
submission.to_csv('submission.csv', index=False)
print("✅ Submission saved successfully!")

Unique image count in submission: 400
Expected image count: 400
✅ Submission saved successfully!
